<a href="https://colab.research.google.com/github/lazaroq11/bill_model/blob/master/bill_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [5]:
!pip install transformers
!pip install -U scikit-learn
!pip install pandas
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 106.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [17]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=0157561f92fd59f80e7b1be30c15f5c4bdb9233a11403ad3ade8ebd6231b4fd1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [8]:
!pip install transformers[torch]
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 7.0 MB/s eta 0:00:00


In [3]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.0 MB/s eta 0:00:00


# ***Treinamento***


In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader

# Seu DataFrame com textos e rótulos
import pandas as pd
import random
import nltk
nltk.download('wordnet')
from textblob import Word

# Seu DataFrame com textos e rótulos
df = pd.DataFrame({
    "label": ["BILL", "BahiaRT", "python", "AI", "Machine Learning", "NLP", "ACSO", "UNEB", "BahiaRt Multidisciplinar", "Yolo", "ROS"],
    "text": ["BILL é um robô de serviço projetado pelo BahiaRT. Ele é um robô humanoide que pode ser usado para uma variedade de tarefas, incluindo atendimento ao cliente, limpeza e segurança. BILL é equipado com uma variedade de sensores e câmeras, que ele usa para navegar pelo ambiente e interagir com humanos. BILL é um produto da BahiaRT, uma equipe de robótica da UNEB. BILL é um dos projetos mais recentes da BahiaRT. BILL é um robô inovador que tem o potencial de melhorar a vida das pessoas. Ele pode ser usado para automatizar tarefas, fornecer assistência aos idosos e deficientes e até mesmo ajudar a salvar vidas.",
            "BahiaRT é uma equipe de robótica da UNEB. A equipe foi fundada em 2007 por um grupo de estudantes e pesquisadores que compartilham o interesse em robótica.",
            "Python é uma linguagem de programação geral de alto nível. É uma linguagem interpretada, o que significa que não precisa ser compilada antes de ser executada. Python é uma linguagem relativamente fácil de aprender e usar.",
            "A inteligência artificial (IA) é uma área da ciência da computação que se concentra no desenvolvimento de sistemas capazes de executar tarefas que normalmente requerem inteligência humana. Ela engloba campos como aprendizado de máquina, visão computacional e processamento de linguagem natural.",
            "A aprendizagem de máquina (Machine Learning) é um subcampo da inteligência artificial que se concentra no desenvolvimento de algoritmos e modelos que permitem que os sistemas aprendam e melhorem com base em dados. É amplamente utilizado em previsões e análise de dados.",
            "O Processamento de Linguagem Natural (NLP) é uma subárea da IA que se concentra na interação entre computadores e linguagem humana. É usado em chatbots, tradução automática e análise de sentimentos, entre outras aplicações.",
            "O ACSO (Núcleo de Arquitetura de Computadores e Sistemas Operacionais), é um grupo de Pesquisa da UNEB, Universidade do Estado da Bahia",
            "A Uneb é uma das maiores universidades da Bahia, multicampi e para todos.",
            "A BahiaRT é uma equipe multidisciplinar que inclui estudantes de engenharia, ciência da computação e design. A BahiaRT tem trabalhado em uma variedade de projetos de robótica, incluindo Bill, um robô de serviço humanoide. A equipe também está trabalhando em um robô de resgate que pode ser usado em situações de emergência. A BahiaRT é uma equipe ativa que está constantemente desenvolvendo novos projetos de robótica. A equipe está comprometida em usar a robótica para melhorar a vida das pessoas.",
            "O Yolo é um algoritmo de AI, utilizado para detecção de pessoas e objetos, BILL utiliza esse algoritmo para executar tarefas relacionadas a isso",
            "Robot Operating System é uma coleção de frameworks de software para desenvolvimento de robôs, que fornece a funcionalidade de um sistema operacional em um cluster de computadores heterogêneo "]
})

# Dividir o DataFrame em treinamento e teste
train_df, test_df = train_test_split(df, test_size=0.2)

# Carregar o tokenizador e modelo pré-treinado GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2LMHeadModel.from_pretrained('gpt2-large')

# Defina um token de padding
tokenizer.pad_token = tokenizer.eos_token

# Defina o tamanho do lote e crie um DataLoader personalizado para treinamento
class TextDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.texts = df['text'].values
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(self.texts[idx], return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        return inputs

train_dataset = TextDataset(train_df, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Configurar otimizador e agendador de taxa de aprendizado
optimizer = AdamW(model.parameters(), lr=2e-4)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * 10)

# Adicione dropout como técnica de regularização
model.config.dropout = 0.3

# Treinamento
model.train()
for epoch in range(10):
    total_loss = 0.0
    for batch in train_dataloader:
        optimizer.zero_grad()
        inputs = batch["input_ids"]
        loss = model(inputs, labels=inputs).loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {total_loss / len(train_dataloader)}')

    # Gere texto de exemplo após algumas épocas
    if (epoch + 1) % 5 == 0:
        model.eval()
        prompt = "BILL"
        prompt_ids = tokenizer(prompt, return_tensors='pt').input_ids
        generated_text = model.generate(prompt_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)
        print(f'Texto gerado na época {epoch + 1}:')
        print(tokenizer.decode(generated_text[0], skip_special_tokens=True))
        model.train()

# Salvar o modelo fine-tunado
model.save_pretrained("fine_tuned_gpt2")
tokenizer.save_pretrained("fine_tuned_gpt2")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ***Métrica Rouge***

In [22]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import pandas as pd
import nltk
from rouge_score import rouge_scorer

nltk.download('punkt')

# Carregar o tokenizador e o modelo pré-treinado GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2LMHeadModel.from_pretrained('fine_tuned_gpt2')  # Carregue o modelo fine-tunado

# Seu DataFrame de teste
test_df = pd.DataFrame({
    "label": ["BILL", "BahiaRT", "python", "AI", "Machine Learning"],
    "text": ["BILL é um robô de serviço projetado pelo BahiaRT. Ele é um robô humanoide que pode ser usado para uma variedade de tarefas.",
             "BahiaRT é uma equipe de robótica da UNEB. A equipe foi fundada em 2007 por um grupo de estudantes e pesquisadores.",
             "Python é uma linguagem de programação geral de alto nível.",
             "A inteligência artificial (IA) é uma área da ciência da computação que se concentra no desenvolvimento de sistemas.",
             "A aprendizagem de máquina (Machine Learning) é um subcampo da inteligência artificial que se concentra no desenvolvimento de algoritmos."],
})

# Função para gerar texto usando o modelo fine-tunado
def generate_text(prompt, model, tokenizer):
    prompt_ids = tokenizer(prompt, return_tensors='pt').input_ids
    generated_text = model.generate(prompt_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)
    return tokenizer.decode(generated_text[0], skip_special_tokens=True)

# Função para calcular as métricas ROUGE
def calculate_rouge_scores(model, tokenizer, test_df):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {"rouge1": {"precision": 0, "recall": 0, "f1": 0},
                    "rouge2": {"precision": 0, "recall": 0, "f1": 0},
                    "rougeL": {"precision": 0, "recall": 0, "f1": 0}}

    for index, row in test_df.iterrows():
        reference_text = row["text"]
        generated_text = generate_text(row["label"], model, tokenizer)

        scores = scorer.score(reference_text, generated_text)

        for metric in ["rouge1", "rouge2", "rougeL"]:
            rouge_scores[metric]["precision"] += scores[metric].precision
            rouge_scores[metric]["recall"] += scores[metric].recall

    for metric in rouge_scores:
        rouge_scores[metric]["precision"] /= len(test_df)
        rouge_scores[metric]["recall"] /= len(test_df)
        p = rouge_scores[metric]["precision"]
        r = rouge_scores[metric]["recall"]
        if p + r > 0:
            f1 = (2 * p * r) / (p + r)
            rouge_scores[metric]["f1"] = f1

    return rouge_scores

# Avaliar o modelo e obter as métricas ROUGE
model.eval()
rouge_scores = calculate_rouge_scores(model, tokenizer, test_df)

print("ROUGE Scores:")
for metric in rouge_scores:
    print(f"{metric}:")
    print(f"Precision: {rouge_scores[metric]['precision']}")
    print(f"Recall: {rouge_scores[metric]['recall']}")
    print(f"F1-score: {rouge_scores[metric]['f1']}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `a

ROUGE Scores:
rouge1:
Precision: 0.5259397163120567
Recall: 0.5204761904761905
F1-score: 0.5231936903681291
rouge2:
Precision: 0.41799906498363715
Recall: 0.3684962406015037
F1-score: 0.3916897734229121
rougeL:
Precision: 0.4992472598323662
Recall: 0.4814285714285715
F1-score: 0.4901760345856269


# ***Inferência***

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Carregar o modelo fine-tunado
fine_tuned_model = GPT2LMHeadModel.from_pretrained("fine_tuned_gpt2")
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("fine_tuned_gpt2")

# Exemplo de inferência com temperatura definida
prompt = "O que é o BILL"
input_ids = fine_tuned_tokenizer.encode(prompt, return_tensors='pt')
temperature = 0.2  # Ajuste o valor de temperatura conforme desejado
max_output_length = 60  # Ajuste o comprimento máximo da saída

with torch.no_grad():
    output = fine_tuned_model.generate(input_ids, max_length=max_output_length, num_return_sequences=1, no_repeat_ngram_size=2, temperature=temperature)
generated_text = fine_tuned_tokenizer.decode(output[0], skip_special_tokens=True)

print("Texto gerado:")
print(generated_text)
